### Situation 1 : singel prompting

In [1]:
%pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.5/725.5 kB 8.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.68.2
    Uninstalling openai-1.68.2:
      Successfully uninstalled openai-1.68.2
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()   # take environment variables from .env.
API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=API_KEY)

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": """1. Extract all the characters, place names, time stamps, and key events from the provided text: “The $25 million settlement paid by President Donald Trump over the Trump University lawsuits continues to impact the 2028 US elections. Candidates are calling for stricter regulations on private universities, referencing the settlement approved by U.S. District Judge Gonzalo Curiel as a landmark victory for students.”.
2. Based on all the real information you know from the internet, assess the factualness of the extracted events. Show your analytic process.
3. Based on all the real information you know from the internet, assess the relationship between all characters, place names, time stamps, and key events. Show your analytic process.
4. Based on your analysis from steps 2 and 3, and all the information you currently have. Does this news article contain any misleading information or mismatched relationships? Show your analytic process and must respond with “0” which represent Not contain any misleading information or “1” which represent contain misleading information."""}],
    stream=False,
)

print(completion.choices[0].message.content)

1. Characters: President Donald Trump, U.S. District Judge Gonzalo Curiel
   Place names: Trump University, 
   Time stamps: 2028 US elections
   Key events: $25 million settlement over Trump University lawsuits, Candidates calling for stricter regulations on private universities, Settlement approved by U.S. District Judge Gonzalo Curiel
   
2. Assessing the factualness:
   - President Donald Trump did indeed settle the Trump University lawsuits for $25 million in 2016. This information is accurate.
   - The 2028 US elections have not occurred yet, so there is no real information to assess the impact of the settlement on those elections. 
   - U.S. District Judge Gonzalo Curiel is a real judge who presided over the Trump University lawsuits and approved the settlement. This information is accurate.
   
3. Relationship analysis:
   - President Donald Trump was involved in the settlement over the Trump University lawsuits.
   - U.S. District Judge Gonzalo Curiel approved the settlement i

### Situation 2 : recursive prompting(start ablation)

In [5]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=API_KEY)

def chatgpt_response(text):
    prompt = f"""1. Extract all the characters, place names, time stamps, and key events from the provided text: "{text}".
2. Based on all the real information you know from the internet, assess the factualness of the extracted events. Show your analytic process.
3. Based on all the real information you know from the internet, assess the relationship between all characters, place names, time stamps, and key events. Show your analytic process.
4. Based on your analysis from steps 2 and 3, and all the information you currently have. Does this news article contain any misleading information or mismatched relationships? Show your analytic process and must respond with "0" which represent Not contain any misleading information or "1" which represent contain misleading information."""
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            stream=False
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        return None

# 讀取Excel檔案
input_file = 'gossipcop-120.xlsx'
output_file = 'ChatGPT_Responses-gossipcop.xlsx'

df = pd.read_excel(input_file)
prompts = df['text'].dropna().tolist()
output_df = pd.DataFrame(columns=df.columns.tolist() + ['ChatGPT Response'])

for i, prompt in enumerate(prompts):
    response = chatgpt_response(prompt)
    new_row = df.iloc[i].tolist() + [response]
    output_df.loc[i] = new_row
    output_df.to_excel(output_file, index=False)
    print(f"Processed {i+1}/{len(prompts)}")

print(f"Processed all prompts and saved responses to {output_file}")

Processed 1/120
Processed 2/120
Processed 3/120
Processed 4/120
Processed 5/120
Processed 6/120
Processed 7/120
Processed 8/120
Processed 9/120
Processed 10/120
Processed 11/120
Processed 12/120
Processed 13/120
Processed 14/120
Processed 15/120
Processed 16/120
Processed 17/120
Processed 18/120
Processed 19/120
Processed 20/120
Processed 21/120
Processed 22/120
Processed 23/120
Processed 24/120
Processed 25/120
Processed 26/120
Processed 27/120
Processed 28/120
Processed 29/120
Processed 30/120
Processed 31/120
Processed 32/120
Processed 33/120
Processed 34/120
Processed 35/120
Processed 36/120
Processed 37/120
Processed 38/120
Processed 39/120
Processed 40/120
Processed 41/120
Processed 42/120
Processed 43/120
Processed 44/120
Processed 45/120
Processed 46/120
Processed 47/120
Processed 48/120
Processed 49/120
Processed 50/120
Processed 51/120
Processed 52/120
Processed 53/120
Processed 54/120
Processed 55/120
Processed 56/120
Processed 57/120
Processed 58/120
Processed 59/120
Proces

In [6]:
import pandas as pd

file_name = "ChatGPT_Responses-gossipcop"

data = pd.read_excel(f'{file_name}.xlsx')
data['Simplified Answer'] = data['Simplified Answer'].astype(str)

# 計算總體準確率（僅考慮 0 和 1）
valid_predictions = data[data['Simplified Answer'].isin(['0', '1'])]
overall_accuracy = (valid_predictions['label'] == valid_predictions['Simplified Answer'].astype(int)).mean()

# 計算 label=1（假新聞）的準確率
label_1_data = valid_predictions[valid_predictions['label'] == 1]
accuracy_label_1 = (label_1_data['label'] == label_1_data['Simplified Answer'].astype(int)).mean()
mistakes_label_1 = len(label_1_data) - (label_1_data['label'] == label_1_data['Simplified Answer'].astype(int)).sum()

# 計算 label=0（真新聞）的準確率
label_0_data = valid_predictions[valid_predictions['label'] == 0]
accuracy_label_0 = (label_0_data['label'] == label_0_data['Simplified Answer'].astype(int)).mean()
mistakes_label_0 = len(label_0_data) - (label_0_data['label'] == label_0_data['Simplified Answer'].astype(int)).sum()

# 統計 "Uncertain" 和 "error" 的情況
uncertain_count = len(data[data['Simplified Answer'] == 'Unknown'])
error_count = len(data[data['Simplified Answer'] == 'ERROR'])

# 計算f1 score
f1_score = 2 * (accuracy_label_1 * accuracy_label_0) / (accuracy_label_1 + accuracy_label_0)

print(f"檔案名稱: {file_name}.xlsx")
print(f"Accuracy: {overall_accuracy:.4f}")
print(f"F1 score: {f1_score:.4f}")
print(f"label=1（假新聞）的準確率: {accuracy_label_1:.2f}, 錯誤數量: {mistakes_label_1}")
print(f"label=0（真新聞）的準確率: {accuracy_label_0:.2f}, 錯誤數量: {mistakes_label_0}")
print(f"Uncertain 的數量: {uncertain_count}")
print(f"ERROR 的數量: {error_count}")


檔案名稱: ChatGPT_Responses-gossipcop.xlsx
Accuracy: 0.8034
F1 score: 0.3291
label=1（假新聞）的準確率: 0.20, 錯誤數量: 16
label=0（真新聞）的準確率: 0.93, 錯誤數量: 7
Uncertain 的數量: 3
ERROR 的數量: 0
